In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from wordcloud import WordCloud

from sklearn.model_selection import KFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
training = pd.read_csv('C:/Users/dlsim/Documents/_College/1 UC Davis/4 Senior Year/Win - STA 142a - Statistical Learning I/ipynb files/142 Final Project/train.csv')
test = pd.read_csv('C:/Users/dlsim/Documents/_College/1 UC Davis/4 Senior Year/Win - STA 142a - Statistical Learning I/ipynb files/142 Final Project/test.csv')
test_labels = pd.read_csv('C:/Users/dlsim/Documents/_College/1 UC Davis/4 Senior Year/Win - STA 142a - Statistical Learning I/ipynb files/142 Final Project/test_labels.csv')

In [ ]:
#training = pd.read_csv("C:/Users/Falak/Desktop/NLP/train.csv")
#test = pd.read_csv("C:/Users/Falak/Desktop/NLP/test.csv")
#test_labels = pd.read_csv("C:/Users/Falak/Desktop/NLP/test_labels.csv")

In [ ]:
#training = pd.read_csv('C:/Users/Alvis/STA142A/train.csv')
#test = pd.read_csv('C:/Users/Alvis/STA142A/test.csv')
#test_labels = pd.read_csv("C:/Users/Alvis/STA142A/test_labels.csv")

In [8]:
###### NEW CODE #####
### Cleaning test data ######

# Getting rid of the rows containing -1s in the test_labels dataset
cleaned_testlabels = test_labels[test_labels != -1]
cleaned_testlabels = cleaned_testlabels.dropna()
cleaned_testlabels.head(20)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,0.0,0.0,0.0,0.0,0.0,0.0
7,000247e83dcc1211,0.0,0.0,0.0,0.0,0.0,0.0
11,0002f87b16116a7f,0.0,0.0,0.0,0.0,0.0,0.0
13,0003e1cccfd5a40a,0.0,0.0,0.0,0.0,0.0,0.0
14,00059ace3e3e9a53,0.0,0.0,0.0,0.0,0.0,0.0
16,000663aff0fffc80,0.0,0.0,0.0,0.0,0.0,0.0
17,000689dd34e20979,0.0,0.0,0.0,0.0,0.0,0.0
19,000844b52dee5f3f,0.0,0.0,0.0,0.0,0.0,0.0
21,00091c35fa9d0465,1.0,0.0,0.0,0.0,0.0,0.0
22,000968ce11f5ee34,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
#converts all values to int
cleaned_testlabels.apply(pd.to_numeric, downcast = 'integer', errors = 'ignore')

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,0,0,0,0,0,0
7,000247e83dcc1211,0,0,0,0,0,0
11,0002f87b16116a7f,0,0,0,0,0,0
13,0003e1cccfd5a40a,0,0,0,0,0,0
14,00059ace3e3e9a53,0,0,0,0,0,0
...,...,...,...,...,...,...,...
153150,fff8f64043129fa2,0,0,0,0,0,0
153151,fff9d70fe0722906,0,0,0,0,0,0
153154,fffa8a11c4378854,0,0,0,0,0,0
153155,fffac2a094c8e0e2,1,0,1,0,1,0


In [15]:
# Joining the test_label dataset to the test dataset
test_with_comments = cleaned_testlabels.join(test.set_index('id'), on='id')

test_with_comments

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text
5,0001ea8717f6de06,0.0,0.0,0.0,0.0,0.0,0.0,Thank you for understanding. I think very high...
7,000247e83dcc1211,0.0,0.0,0.0,0.0,0.0,0.0,:Dear god this site is horrible.
11,0002f87b16116a7f,0.0,0.0,0.0,0.0,0.0,0.0,"""::: Somebody will invariably try to add Relig..."
13,0003e1cccfd5a40a,0.0,0.0,0.0,0.0,0.0,0.0,""" \n\n It says it right there that it IS a typ..."
14,00059ace3e3e9a53,0.0,0.0,0.0,0.0,0.0,0.0,""" \n\n == Before adding a new product to the l..."
...,...,...,...,...,...,...,...,...
153150,fff8f64043129fa2,0.0,0.0,0.0,0.0,0.0,0.0,":Jerome, I see you never got around to this…! ..."
153151,fff9d70fe0722906,0.0,0.0,0.0,0.0,0.0,0.0,==Lucky bastard== \n http://wikimediafoundatio...
153154,fffa8a11c4378854,0.0,0.0,0.0,0.0,0.0,0.0,==shame on you all!!!== \n\n You want to speak...
153155,fffac2a094c8e0e2,1.0,0.0,1.0,0.0,1.0,0.0,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...


In [19]:
# Preprocessing
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')

def preprocess(training, test):
    training['clean'] = training.apply(lambda row: row['comment_text'].replace("\n"," "), axis=1) #removes new line character
    test['clean'] = test.apply(lambda row: row['comment_text'].replace("\n"," "), axis=1)

    #removes urls
    training['clean'] = training.apply(lambda row: re.sub('http://\S+|https://\S+', '', row['clean']), axis=1)
    test['clean'] = test.apply(lambda row: re.sub('http://\S+|https://\S+', '',row['clean']), axis=1)

    #remove all non-alphanumeric values
    training['clean'] = training.apply(lambda row: re.sub('[^A-Za-z0-9\' ]+', '',row['clean']), axis=1)
    test['clean'] = test.apply(lambda row: re.sub('[^A-Za-z0-9\' ]+', '',row['clean']), axis=1)

    #remove stopwords as they occupy major chunk of the vocabulary
    training['clean'] = training['clean'].apply(lambda x: ' '.join([word for word in x.split() if not word in (stop)]))
    test['clean'] = test['clean'].apply(lambda x: ' '.join([word for word in x.split() if not word in (stop)]))

    #removes all additional spaces
    training['clean'] = training.apply(lambda row: re.sub(' +', ' ', row['clean']).strip(), axis=1)
    test['clean'] = test.apply(lambda row: re.sub(' +', ' ', row['clean']).strip(), axis=1)
    
    #lowercase end result
    training['clean'] = training['clean'].str.lower()
    test = test['clean'].str.lower()

preprocess(training, test_with_comments)
training.head(10)
test.head(10)

,id,comment_text,clean
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,Yo bitch Ja Rule succesful ever whats hating s...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,From RfC The title fine IMO
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",Sources Zawe Ashton Lapland
3,00017563c3f7919a,":If you have a look back at the source, the in...",If look back source information I updated corr...
4,00017695ad8997eb,I don't anonymously edit articles at all.,I anonymously edit articles
5,0001ea8717f6de06,Thank you for understanding. I think very high...,Thank understanding I think highly would rever...
6,00024115d4cbde0f,Please do not add nonsense to Wikipedia. Such ...,Please add nonsense Wikipedia Such edits consi...
7,000247e83dcc1211,:Dear god this site is horrible.,Dear god site horrible
8,00025358d4737918,""" \n Only a fool can believe in such numbers. ...",Only fool believe numbers The correct number l...
9,00026d1092fe71cc,== Double Redirects == \n\n When fixing double...,Double Redirects When fixing double redirects ...


In [20]:
## Checking accuracy score for toxic label
X_train = training
X_test = test_with_comments
Y_train = X_train['toxic']

#Vectorization of training cleaned comments
tfidf_vectorizer = TfidfVectorizer(use_idf=True, max_df=0.7) 
tfidf_vectorizer.fit_transform(X_train['clean'].values)

train_feature_set=tfidf_vectorizer.transform(X_train['clean'].values)
test_feature_set=tfidf_vectorizer.transform(X_test['clean'].values)

#Model build using training data cleaned comments
scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)
model = scikit_log_reg.fit(train_feature_set, Y_train) 

Y_predicted = model.predict(test_feature_set) #Y_predicted are the predicted labels from our model
Y_predicted_prob = model.predict_proba(test_feature_set) #proba model shows two columns: 1st column is prob that it is classified as 0, 2nd column is prob that it is classified as 1
#print(Y_predicted_prob)

Y_validation = X_test['toxic']
accuracy_score(Y_validation, Y_predicted)

[LibLinear]

0.9299759292256713

In [21]:
### Checking accuracy scores for each label


for i in range(1,7):
    X_train = training
    X_test = test_with_comments
    Y_train = X_train.iloc[:,i+1]

    tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.7) #Vectorization of training cleaned comments
    tfidf_vectorizer.fit_transform(X_train['clean'].values)

    train_feature_set=tfidf_vectorizer.transform(X_train['clean'].values)
    test_feature_set=tfidf_vectorizer.transform(X_test['clean'].values)

    scikit_log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)
    model = scikit_log_reg.fit(train_feature_set, Y_train) #Model build using training data cleaned comments

    Y_predicted = model.predict(test_feature_set) #Y_predicted are the predicted labels from our model
    Y_predicted_prob = model.predict_proba(test_feature_set) #proba model shows two columns: 1st column is prob that it is classified as 0, 2nd column is prob that it is classified as 1
    #print(proba)
 
    #Predicted Accuracy score

    Y_test = X_test.iloc[:,i]
    print(f"The classification accuracy of the fitted model for the {test_with_comments.columns[i]} label is {accuracy_score(Y_test, Y_predicted)}")

[LibLinear]The classification accuracy of the fitted model for the toxic label is 0.9299759292256713
[LibLinear]The classification accuracy of the fitted model for the severe_toxic label is 0.9931382662790335
[LibLinear]The classification accuracy of the fitted model for the obscene label is 0.9653474631904717


KeyboardInterrupt: 